In [1]:
import pandas as pd

pd.options.plotting.backend = "plotly"
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots


import plotly.io as pio
import plotly.express as px

pio.templates.default = "simple_white"

In [2]:
time_taken = pd.Series([1174.0,
573.0,
358.0,
983.0,
455.0,
242.0,
255.0,
389.0,
323.0,
465.0,
436.0,
1094.0,
283.0,
947.0,
291.0,
719.0,
364.0,
343.0,
185.0,
271.0,
685.0,
342.0,
1752.0,
215.0,
2324.0,
747.0,
571.0,
477.0,
654.0,
288.0,
419.0,
235.0,
347.0,
785.0,
678.0,
716.0,
474.0,
334.0,
891.0,
268.0,
271.0,
652.0,
344.0,
170.0,
249.0,
225.0,
349.0,
231.0,
376.0,
486.0,
875.0,
298.0,
269.0,
801.0,
295.0,
635.0,
370.0,
145.0,
698.0,
409.0,
678.0,
282.0,
305.0,
372.0,
465.0,
639.0,
202.0,
285.0,
627.0,
364.0,
792.0,
616.0,
398.0,
473.0,
647.0,
332.0,
1656.0,
202.0,
279.0,
835.0,
678.0,
157.0,
216.0,
693.0,
261.0,
241.0,
275.0,
542.0,
456.0,
383.0,
446.0,
377.0,
332.0,
399.0,
309.0,
585.0,
670.0,
420.0]
)

In [3]:
time_taken.plot.hist()

In [4]:
mean_std_series = pd.Series(
    [(x - time_taken.mean()) / time_taken.std() for x in time_taken.to_list()]
)
mean_std_series.loc[
    mean_std_series < mean_std_series.mean() - 3 * mean_std_series.std()
]

Series([], dtype: float64)

In [5]:
mean_std_series.hist()

In [6]:
data_usa_raw = pd.read_csv("../data/raw/usa/all_apps_wide_2023-11-14_USA.csv")
data_deu_1_raw = pd.read_csv("../data/raw/deutsch/all_apps_wide_2023-11-14_DEU.csv")
data_deu_2_raw = pd.read_csv(
    "../data/raw/deutsch runde 2/all_apps_wide_2023-11-15_DEU.csv"
)

In [7]:
relevant_columns = [
    "survey.1.player.positive_szenarios",
    "survey.1.player.treatment_pa",
    "survey.1.player.PRS1T1",
    "survey.1.player.PRS1T2",
    "survey.1.player.PRS2T1",
    "survey.1.player.PRS2T2",
    "survey.1.player.PRS3T1",
    "survey.1.player.PRS3T2",
    "survey.1.player.PRS4T1",
    "survey.1.player.PRS4T2",
    "survey.1.player.FRS1T1",
    "survey.1.player.FRS1T2",
    "survey.1.player.FRS2T1",
    "survey.1.player.FRS2T2",
    "survey.1.player.FRS3T1",
    "survey.1.player.FRS3T2",
    "survey.1.player.FRS4T1",
    "survey.1.player.FRS4T2",
    "survey.1.player.ARS1T1",
    "survey.1.player.ARS1T2",
    "survey.1.player.ARS2T1",
    "survey.1.player.ARS2T2",
    "survey.1.player.ARS3T1",
    "survey.1.player.ARS3T2",
    "survey.1.player.ARS4T1",
    "survey.1.player.ARS4T2",
    "survey.1.player.ETHS1T1",
    "survey.1.player.ETHS1T2",
    "survey.1.player.ETHS2T1",
    "survey.1.player.ETHS2T2",
    "survey.1.player.ETHS3T1",
    "survey.1.player.ETHS3T2",
    "survey.1.player.ETHS4T1",
    "survey.1.player.ETHS4T2",
    "survey.1.player.PROS1T1",
    "survey.1.player.PROS1T2",
    "survey.1.player.PROS2T1",
    "survey.1.player.PROS2T2",
    "survey.1.player.PROS3T1",
    "survey.1.player.PROS3T2",
    "survey.1.player.PROS4T1",
    "survey.1.player.PROS4T2",
    "origin"
]

In [8]:
data_all = (
    pd.concat(
        [
            data_usa_raw.assign(origin="USA"),
            data_deu_1_raw.assign(origin="DEU"),
            data_deu_2_raw.assign(origin="DEU"),
        ],
        axis=0,
    )
    .loc[lambda df_: df_["participant._current_page_name"] == "End"]
    .loc[:, relevant_columns]
    .rename(
        columns=dict(
            zip(
                relevant_columns,
                [x.replace("survey.1.player.", "") for x in relevant_columns],
            )
        )
    )
)

data_usa = data_all.loc[data_all.origin == "USA"]
data_deu = data_all.loc[data_all.origin == "DEU"]

Scale explanation:
- FRS1T1 -->  feel responsible senario 1, team lead 1 (familiar solution)
- ARS1T1 -->  act responsible senario 1, team lead 1 (familiar solution)
- FRS1T2 -->  feel responsible senario 1, team lead 2 (new solution)
- ARS1T2 -->  act responsible senario 1, team lead 2 (new solution)

In [9]:
scenario_dict_outcome = {
    "1": {"positive": "1_and_4", "negative": "2_and_3"},
    "2": {"positive": "2_and_3", "negative": "1_and_4"},
    "3": {"positive": "2_and_3", "negative": "1_and_4"},
    "4": {"positive": "1_and_4", "negative": "2_and_3"},
}

scenario_dict_pa = {
    "1": {"yes": "1_and_4", "no": "2_and_3"},
    "2": {"yes": "2_and_3", "no": "1_and_4"},
    "3": {"yes": "2_and_3", "no": "1_and_4"},
    "4": {"yes": "1_and_4", "no": "2_and_3"},
}


def get_mean_scales(data, scale, scenario_nr, solution_type, outcome, pa):
    scale_string = ""
    if scale == "feel":
        scale_string += "FR"
    elif scale == "act":
        scale_string += "AR"
    elif scale == "risk":
        scale_string += "PR"

    scale_string += f"S{scenario_nr}"

    if solution_type == "familiar":
        scale_string += "T1"
    else:
        scale_string += "T2"

    outcome_filter = scenario_dict_outcome[str(scenario_nr)][outcome]
    pa_filter = scenario_dict_pa[str(scenario_nr)][pa]

    return data.loc[
        (data.positive_szenarios == outcome_filter)
        & (data.treatment_pa == pa_filter),
        scale_string,
    ].mean()

## Plot Fig 1 from Nordbye

In [10]:
x_axis_values_list = [
    ["Familiar solution", "New solution"],
    ["Follow advise", "Not follow advise"],
    ["Hold on", "Change decision"],
    ["Wait and see", "Take action"],
]

d3_colors = px.colors.qualitative.D3
colors_dict = {
    "feel responsible + positive outcome + PA yes": d3_colors[0],
    "feel responsible + positive outcome + PA no": d3_colors[1],
    "feel responsible + negative outcome + PA yes": d3_colors[2],
    "feel responsible + negative outcome + PA no": d3_colors[3],
    "act responsible + positive outcome + PA yes": d3_colors[4],
    "act responsible + positive outcome + PA no": d3_colors[5],
    "act responsible + negative outcome + PA yes": d3_colors[6],
    "act responsible + negative outcome + PA no": d3_colors[7],
}


def create_line_subplot(data, output_file_name, show_fig, fig_title):
    fig = make_subplots(rows=2, cols=2)

    for scenario, x_axis_values in zip(range(1, 5), x_axis_values_list):
        for responsibility_scale in ["feel", "act"]:
            for outcome in ["positive", "negative"]:
                for pa_available in ["yes", "no"]:
                    fig.add_trace(
                        go.Scatter(
                            x=x_axis_values,
                            y=[
                                get_mean_scales(
                                    data=data,
                                    scale=responsibility_scale,
                                    scenario_nr=scenario,
                                    solution_type="familiar",
                                    outcome=outcome,
                                    pa=pa_available,
                                ),
                                get_mean_scales(
                                    data=data,
                                    scale=responsibility_scale,
                                    scenario_nr=scenario,
                                    solution_type="new",
                                    outcome=outcome,
                                    pa=pa_available,
                                ),
                            ],
                            mode="lines+markers",
                            name=f"{responsibility_scale} responsible + {outcome} outcome + PA {pa_available}",
                            legendgroup=f"{responsibility_scale} responsible + {outcome} outcome + PA {pa_available}",
                            marker_color=colors_dict[
                                f"{responsibility_scale} responsible + {outcome} outcome + PA {pa_available}"
                            ],
                            showlegend=True if scenario == 1 else False,
                        ),
                        row=1 if scenario in [1, 2] else 2,
                        col=1 if scenario in [1, 3] else 2,
                    )
    fig.update_yaxes(range=[1, 7], dtick=1)
    fig.update_layout(width=900 + 200, height=600 + 200, title=fig_title)
    fig.write_html(f"{output_file_name}.html")
    if show_fig:
        fig.show()

In [11]:
create_line_subplot(data=data_all, output_file_name="scenarios_all", show_fig=True, fig_title="All")

In [12]:
create_line_subplot(
    data=data_deu, output_file_name="scenarios_deu", show_fig=True, fig_title="DEU"
)

In [13]:
create_line_subplot(
    data=data_usa, output_file_name="scenarios_usa", show_fig=True, fig_title="USA"
)

## who is perceied to feel / act more responsible?

In [28]:
# who is perceied to feel more responsible?
t1_columns_feel = [f"FRS{x}T1" for x in range(1, 5)]
t2_columns_feel = [f"FRS{x}T2" for x in range(1, 5)]

# who is perceied to act more responsible?
t1_columns_act = [f"ARS{x}T1" for x in range(1, 5)]
t2_columns_act = [f"ARS{x}T2" for x in range(1, 5)]

pd.DataFrame(
    {
        "t1_feel": data_all[t1_columns_feel].to_numpy().reshape(-1),
        "t2_feel": data_all[t2_columns_feel].to_numpy().reshape(-1),
        "t1_act": data_all[t1_columns_act].to_numpy().reshape(-1),
        "t2_act": data_all[t2_columns_act].to_numpy().reshape(-1),
    }
).describe()

,t1_feel,t2_feel,t1_act,t2_act
count,680.000000,680.000000,680.000000,680.000000
mean,4.786765,5.948529,5.213235,4.685294
std,1.887124,1.290727,1.566438,1.632085
min,1.000000,1.000000,1.000000,1.000000
25%,3.000000,5.000000,4.000000,4.000000
50%,5.000000,6.000000,6.000000,5.000000
75%,6.000000,7.000000,6.000000,6.000000
max,7.000000,7.000000,7.000000,7.000000


--> t2 (B) will feel more responsible and acted less responsible

## Regression

In [14]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [24]:
scenario_dict_pa
# scenario_dict_outcome

{'1': {'yes': '1_and_4', 'no': '2_and_3'},
 '2': {'yes': '2_and_3', 'no': '1_and_4'},
 '3': {'yes': '2_and_3', 'no': '1_and_4'},
 '4': {'yes': '1_and_4', 'no': '2_and_3'}}

In [45]:
def get_regression_df(szenario_nr, scale):
    regression_df = (
        data_all[["positive_szenarios", "treatment_pa", scale]]
        .assign(
            outcome_positive=data_all.positive_szenarios.replace(
                scenario_dict_outcome[str(szenario_nr)]["positive"], 1
            ).replace(scenario_dict_outcome[str(szenario_nr)]["negative"], 0),
            pa_available=data_all.treatment_pa.replace(
                scenario_dict_pa[str(szenario_nr)]["yes"], 1
            ).replace(scenario_dict_pa[str(szenario_nr)]["no"], 0),
            participant=range(data_all.shape[0]),
        )
        .loc[:, [scale, "outcome_positive", "pa_available", "participant"]]
    )
    return regression_df

In [76]:
data_all.columns

Index(['positive_szenarios', 'treatment_pa', 'PRS1T1', 'PRS1T2', 'PRS2T1',
       'PRS2T2', 'PRS3T1', 'PRS3T2', 'PRS4T1', 'PRS4T2', 'FRS1T1', 'FRS1T2',
       'FRS2T1', 'FRS2T2', 'FRS3T1', 'FRS3T2', 'FRS4T1', 'FRS4T2', 'ARS1T1',
       'ARS1T2', 'ARS2T1', 'ARS2T2', 'ARS3T1', 'ARS3T2', 'ARS4T1', 'ARS4T2',
       'ETHS1T1', 'ETHS1T2', 'ETHS2T1', 'ETHS2T2', 'ETHS3T1', 'ETHS3T2',
       'ETHS4T1', 'ETHS4T2', 'PROS1T1', 'PROS1T2', 'PROS2T1', 'PROS2T2',
       'PROS3T1', 'PROS3T2', 'PROS4T1', 'PROS4T2', 'origin'],
      dtype='object')

In [85]:
scenario = 1
dependent_variable = "AR"

t1_df = (
    data_all[
        ["positive_szenarios", "treatment_pa", f"{dependent_variable}S{scenario}T1"]
    ]
    .assign(
        outcome_positive=data_all.positive_szenarios.replace(
            scenario_dict_outcome[str(scenario)]["positive"], 1
        ).replace(scenario_dict_outcome[str(scenario)]["negative"], 0),
        pa_available=data_all.treatment_pa.replace(
            scenario_dict_pa[str(scenario)]["yes"], 1
        ).replace(scenario_dict_pa[str(scenario)]["no"], 0),
        participant=range(data_all.shape[0]),
        active_teamlead="yes",
    )
    .rename(columns={f"{dependent_variable}S{scenario}T1": dependent_variable})
)

t2_df = (
    data_all[
        ["positive_szenarios", "treatment_pa", f"{dependent_variable}S{scenario}T2"]
    ]
    .assign(
        outcome_positive=data_all.positive_szenarios.replace(
            scenario_dict_outcome[str(scenario)]["positive"], 1
        ).replace(scenario_dict_outcome[str(scenario)]["negative"], 0),
        pa_available=data_all.treatment_pa.replace(
            scenario_dict_pa[str(scenario)]["yes"], 1
        ).replace(scenario_dict_pa[str(scenario)]["no"], 0),
        participant=range(data_all.shape[0]),
        active_teamlead="no",
    )
    .rename(columns={f"{dependent_variable}S{scenario}T2": dependent_variable})
)

(
    pd.concat([t1_df, t2_df], axis=0).loc[
        :,
        [
            dependent_variable,
            "outcome_positive",
            "pa_available",
            "active_teamlead",
            "participant",
        ],
    ]
)

,positive_szenarios,treatment_pa,AR,outcome_positive,pa_available,participant,active_teamlead
0,1_and_4,1_and_4,7.0,1,1,0,yes
1,2_and_3,1_and_4,5.0,0,1,1,yes
2,1_and_4,2_and_3,6.0,1,0,2,yes
4,1_and_4,1_and_4,7.0,1,1,3,yes
5,2_and_3,1_and_4,5.0,0,1,4,yes
...,...,...,...,...,...,...,...
36,1_and_4,1_and_4,6.0,1,1,165,no
37,2_and_3,1_and_4,6.0,0,1,166,no
38,1_and_4,2_and_3,5.0,1,0,167,no
39,2_and_3,2_and_3,5.0,0,0,168,no


In [69]:
scenario = 1
dependent_variable = "AR"

regression_df = get_regression_df(1, dependent_variable)

mod = smf.ols(
    formula=f"{dependent_variable} ~ C(outcome_positive) + C(pa_available) + 1|participant",
    data=regression_df,
)
res = mod.fit()
res.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,4.4302,0.343,12.920,0.000,3.753,5.107
C(outcome_positive)[T.1],0.2442,0.281,0.869,0.386,-0.310,0.799
C(pa_available)[T.1],0.1730,0.280,0.618,0.538,-0.380,0.726
1 | participant,0.0032,0.003,1.108,0.269,-0.002,0.009


## Vignette 1 -- Choosing a familiar or new solution

In [ ]:
## what is perceived more risky?

get_mean_scales(
    data=data_all,
    responsibility_scale="risk",
    scenario_nr=1,
    solution_type="familiar",
    outcome=outcome,
    pa=pa_available,
)